# fin article 1

## On le fait pour le daily News 

Dans ce NB, après avoir trouvé les mots les plus significatifs qui donnemt des infos sur l'orientation politique des journaux. Nous allons vérifier si notre modéle réussit à trouver l'orientation politique de the guardian et de daily_news

In [88]:
import pandas as pd 
from src.utils import * 
from src.data_processing import * 

pd.set_option('display.max_columns', None)

Un pretraitement a déjà été effectué. Tous les articles dans ce dataframe portent sur les sujets de la bigtech

In [89]:
# df_daily_express = pd.read_csv('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv', encoding='utf-8')
# df_daily_express = read_input('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv', encod='utf-8')

In [90]:
df_daily_express = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv', encoding='utf-8')

In [91]:
# significant_words = read_input('01. output/df_500_with_slope_coef.csv')
significant_words = pd.read_csv('01. output/df_500_with_slope_coef.csv', encoding='utf-8')

In [92]:
# on quitte la vision 
text = df_daily_express['fulltext'].sum()
len(text)

52213541

In [93]:
# On est obligé de faire un découpage à cause des limites des fcts de cleening sur spacy
text_1 = text[:1000000]
text_2 = text[1000000:2000000]
text_3 = text[2000000:3000000]
dico_text = {}
dico_text_clean = {}
n = len(text) // 1000000
for i in range(n+1):
    debut = i*1000000
    fin = (i+1)*1000000
    dico_text[i] = text[debut:fin]
    dico_text_clean[i] = clean(dico_text[i])
text = []
for morceau in dico_text_clean.values():
    text += morceau


/Users/ismailakrout/opt/anaconda3/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [94]:
def count_liste(lili, mot):
    return lili.count(mot)

In [95]:
liste_of_qi2_words = significant_words.columns[5:]

In [96]:
significant_words

,Unnamed: 0,Speaker,party,text,party_bool,hon,gentleman,govern,ladi,absolut,european,labour,agre,deliv,constitu,uk,commit,â£,recognis,money,union,debat,welcom,fund,amend,discuss,improv,nhs,kingdom,protect,law,secur,reform,consid,encourag,requir,benefit,eu,lord,bbc,job,sector,parliament,duti,oper,concern,review,technolog,legisl,regul,vote,effect,spend,measur,introduc,data,provis,inde,appli,britain,offenc,specif,addit,programm,propos,individu,wish,success,elect,claus,econom,congratul,obvious,shall,grate,inform,learn,progress,patient,enabl,agreement,proper,subject,awar,refer,england,therefor,organis,polit,independ,achiev,complet,court,respect,northern,ireland,excel,approach,exist,vital,delight,apprenticeship,shadow,sort,activ,practic,colleagu,perhap,g,exact,speaker,crime,beer,educ,extrem,scotland,commend,employ,hear,mention,negoti,assur,campaign,referendum,promot,suggest,yes,carri,polici,design,contribut,tribut,necessari,reassur,crimin,extra,serv,rural,fair,appropri,accept,arrang,scheme,greater,pub,enterpris,dog,innov,engin,scottish,control,chariti,fulli,respond,engag,product,conduct,listen,basi,trust,initi,reduc,common,train,scienc,deficit,welfar,confid,commiss,defenc,regim,maintain,investig,strategi,circumst,consum,standard,principl,consult,purpos,ministeri,rememb,prosecut,determin,object,mobil,argument,broadband,consider,longterm,midland,presid,speech,figur,resid,proud,proceed,generat,establish,border,coverag,cover,gas,skill,environ,export,unemploy,prison,repres,reflect,wind,incred,infrastructur,implement,safe,abil,green,relev,sentenc,cornwal,highlight,solut,taxpay,natur,competit,remind,answer,balanc,freedom,framework,pass,citizen,extend,simpli,regist,syria,attract,flexibl,draft,partnership,intervent,remov,sir,advic,rout,produc,remark,connect,communic,appoint,tackl,network,identifi,judg,perfect,penalti,intent,intend,link,commission,prioriti,chancellor,arm,cultur,drug,wale,applic,electr,word,section,outcom,custom,justic,inquiri,enforc,essenti,procedur,stalk,owner,price,statutori,treati,aim,terrorist,throughout,cancer,send,transpar,guidanc,sensibl,devolv,doubt,highway,recommend,choic,notic,urg,whom,approv,vehicl,regular,prepar,paper,elector,exercis,assist,strike,enjoy,inherit,safeguard,precis,sit,capabl,save,constitut,pupil,partner,treat,boost,enhanc,function,wide,prevent,parliamentari,pleasur,financ,oversea,oil,modern,oblig,coalit,judgment,combin,conserv,behaviour,regard,peer,affect,nuclear,deploy,commerci,strengthen,aid,celebr,illeg,suspect,news,qualiti,keen,option,correct,code,aspect,internet,licenc,institut,fantast,behalf,write,effici,sometim,debt,mental,chamber,christchurch,advantag,threat,pedlar,passion,stafford,emerg,danger,expand,consist,treatment,premis,ncs,appreci,capac,weapon,tabl,westminst,motion,everybodi,instanc,elsewher,conclus,broad,treasuri,manufactur,ultim,wider,suppli,target,express,brexit,sustain,charter,intellig,champion,screen,payment,restrict,immigr,maximum,construct,invit,search,consent,draw,manifesto,content,occas,lesson,particip,robust,damag,enorm,leadership,politician,convict,primari,ofcom,retain,transport,frank,type,slight,critic,deliveri,fundament,russia,normal,locat,crown,histor,honour,appal,transit,bureaucraci,convers,anim,condit,mess,somerset,snp,agenda,equal,status,medicin,necessarili,yesterday,incom,demonstr,knowledg,sad,profession,mechan,bench,privileg,desir,clinic,size,stick,altern,societi,royal,alongsid,attend,beauti,technic,music,permiss,commonwealth,corpor,ongo,excit,abus,background,alleg,reduct,legitim,easier,youth,fee,resourc,tool,choos,prosper,facil,expertis,equip,supermarket,afghanistan,valuabl,mayor,crucial,villag,dorset,onshor,spoken,borough,bear,otherwis,renew
0,0,Adam Afriyie,Con,"['scienc', 'technolog', 'facil', 'grant', 'fel...",1.0,0.046602,0.010680,0.053398,0.001942,0.001942,0.003883,0.005825,0.005825,0.003883,0.014563,0.009709,0.002913,0.004854,0.009709,0.001942,0.001942,0.009709,0.005825,0.004854,0.002913,0.001942,0.001942,0.004854,0.001942,0.000971,0.001942,0.002913,0.000000,

In [97]:
df_journal = pd.DataFrame([[word, count_liste(text, word), significant_words.loc[865, word],  significant_words.loc[866, word]] for word in liste_of_qi2_words], columns=['mot', 'freq', 'intercept', 'slope_coefficient'])



In [98]:
n = df_journal['freq'].sum()

In [99]:
df_journal['freq_relatif'] = df_journal['freq']/n 

In [100]:
df_journal.head(50)

,mot,freq,intercept,slope_coefficient,freq_relatif
0,hon,30,0.043510,0.011250,0.000051
1,gentleman,18,0.008169,-0.000971,0.000031
2,govern,6755,0.045267,-0.015173,0.011561
3,ladi,182,0.001818,0.002568,0.000311
4,absolut,643,0.003812,0.000260,0.001100
5,european,2563,0.003290,0.001358,0.004386
6,labour,826,0.005750,-0.002003,0.001414
7,agre,1632,0.015384,-0.000433,0.002793
8,deliv,1517,0.003279,0.001265,0.002596
9,constitu,127,0.018972,-0.003965,0.000217


In [101]:
import statsmodels.api as sm

y = df_journal['freq_relatif'] - df_journal['intercept']
x = df_journal['slope_coefficient']

# adding the constant term
x = sm.add_constant(x)

result = sm.OLS(y, x).fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     15.68
Date:                Thu, 09 Feb 2023   Prob (F-statistic):           8.61e-05
Time:                        18:34:46   Log-Likelihood:                 2100.2
No. Observations:                 500   AIC:                            -4196.
Df Residuals:                     498   BIC:                            -4188.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              6.443e-06      0.00

## On le fait pout the guardian 

In [102]:
df_the_guardian = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv', encoding='utf-8')


In [103]:
df_the_guardian[]

SyntaxError: invalid syntax (3463501004.py, line 1)